In [1]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import pickle
import os
import glob

In [13]:
df = pd.read_hdf("D:\Mtech\Sem4\ASR\Project\\test_timit.hdf")
features = np.array(df["features"].tolist())
label = np.array(df["labels"].tolist())
df1 = pd.DataFrame(features)
df2 = pd.DataFrame(label)
df = pd.merge(df1, df2, right_index=True, left_index=True)
df.rename(columns={'0_y': 'Label'}, inplace=True)
print(df.head())

        0_x          1         2         3         4         5         6  \
0  6.900333 -31.086409 -2.435996 -2.776463 -0.808890 -3.903032  4.633207   
1  7.047228 -33.148942 -4.233357 -6.645017  0.904536  1.434145  8.700283   
2  7.244530 -31.643784  0.315534 -3.996243  2.608060  0.930876  5.632338   
3  7.324972 -29.570619  2.366958 -2.780892  0.755487 -4.894153  5.710508   
4  7.321366 -30.823434  0.840167 -4.758817 -0.360900 -5.599111  5.115698   

          7         8         9        10        11        12 Label  
0  3.699078  0.218039  1.904689 -3.646656 -3.641317 -2.606387   sil  
1  9.506688  1.032303  6.516931  8.285855  4.784435  5.485925   sil  
2  4.911666 -2.691708 -1.599512  3.772963  2.246570  6.085565   sil  
3  2.231653  1.616013 -5.261802  0.721313 -3.738060 -0.459576   sil  
4  2.199870  4.351945 -5.062879  7.001394  5.654462  5.851408   sil  


In [14]:
perCt =0
totalCt =0

def calPER(x,lbl):
    global perCt
    global totalCt
    totalCt += 1
    if x[lbl] != x['Label']:
        #print(x[lbl],x['Label'])
        if(len(x[lbl]) == 0 and len(x['Label'])!= 0 ) or (len(x[lbl]) != 0 and len(x['Label'])== 0 ) or (len(x[lbl]) != 0 and len(x['Label'])!= 0 ):
            perCt += 1

        
test_df = df.iloc[:,:-1]
for i in range(1,8):
    filename = ''.join(['D:\Mtech\Sem4\ASR\Project\Model\\',str(2**i),'M_GMM\\','*.pkl'])
    eval_df = pd.DataFrame()
    for file in glob.glob(filename):
        #print(file)
        with open(file, "rb") as f:
            gmm = pickle.load(f)
            score = gmm.score_samples(test_df)
            label = os.path.basename(file).split('_')[0]
            eval_df[label] = score
    eval_label = ''.join(['eval_score_',str(2**i),'M_GMM'])
    df[eval_label] = eval_df.idxmax(axis=1)
    df.apply(calPER,axis=1,args=(eval_label,))
    print('PER for',str(2**i),'M_GMM',perCt/totalCt)
    perCt =0
    totalCt =0


PER for 2 M_GMM 0.8645264136740025
PER for 4 M_GMM 0.8535845547535758
PER for 8 M_GMM 0.8443718726475667
PER for 16 M_GMM 0.8489771066731612
PER for 32 M_GMM 0.8504228844706195
PER for 64 M_GMM 0.8536288358499756
PER for 128 M_GMM 0.8573263073993712
